## I will use Jupyter notebook to present my second example **HTMLgen**
First, review of basics of Python's object model.

In [70]:
class Student:
    def __init__(self, name):
        self.name = name

In [71]:
ethan = Student('Ethan')

In [72]:
ethan.name

'Ethan'

In [73]:
# ethan.citizenship

In [74]:
mary = Student('Mary')

In [75]:
mary.name

'Mary'

In [76]:
# mary.age

In [77]:
class Student:
    citizenship = 'US'
    def __init__(self, name):
        self.name = name

In [78]:
eric = Student('Eric')

In [79]:
eric.name

'Eric'

In [80]:
vars(eric)

{'name': 'Eric'}

In [81]:
eric.citizenship

'US'

In [82]:
pwd

'/Users/lubyliao/cs305'

In [83]:
cd cs305

[Errno 2] No such file or directory: 'cs305'
/Users/lubyliao/cs305


# First version of Href-gen: 
1. CS150 style 
1. All arguments are required
1. Initializer simply uses arguments provided to eastablish the state of the Href instance
1. All Href instances have the same set of six attributes

In [84]:
cat H_1.py

class Href:

    """The first Href class.  Totally strightforward.  Totally boring
    """

    def __init__(self,
                 url,
                 text,
                 target,
                 onClick,
                 onMouseOver,
                 onMouseOut):
        self.url = url
        self.text = text
        self.target = target
        self.onClick = onClick 
        self.onMouseOver = onMouseOver
        self.onMouseOut = onMouseOut 


    def __str__(self):
        s = ['<A HREF="%s"' % self.url]
        if self.target: s.append(' TARGET="%s"' % self.target)
        if self.onClick: s.append(' onClick="%s"' % self.onClick)
        if self.onMouseOver: s.append(' onMouseOver="%s"' % self.onMouseOver)
        if self.onMouseOut: s.append(' onMouseOut="%s"' % self.onMouseOut)
        s.append('>%s</A>' % self.text)
        return ''.join(s)

A = Href

usd = Href('http://www.sandiego.edu',
            'USD',
            None,
            None,
            None,
        

In [85]:
run H_1.py

<A HREF="http://www.sandiego.edu">USD</A>
<A HREF="http://mit.edu" TARGET="_blank">MIT</A>


In [86]:
vars(usd)

{'onClick': None,
 'onMouseOut': None,
 'onMouseOver': None,
 'target': None,
 'text': 'USD',
 'url': 'http://www.sandiego.edu'}

In [87]:
vars(mit)

{'onClick': None,
 'onMouseOut': None,
 'onMouseOver': None,
 'target': '_blank',
 'text': 'MIT',
 'url': 'http://mit.edu'}

In [88]:
print(len(vars(usd))); print(len(vars(mit)))

6
6


##### Even though clients are forces to provide all required arguments, they do not need to provide them positionally, which is very hard and error prone when the arguments are numerous.  This is because we may remember what arguments are required, but who can remember in what order they should be provided?   For example, we know that to create a link, we must provide the URL and link text, but I could never remember which comes first.

Here is how you can call the initilizer by using keyword arguments:

In [89]:
ucla = Href(onClick=None,
            onMouseOut=None,
            onMouseOver=None,
            target=None,
            text='UCLA',
            url='http://www.ucla.edu')
print(ucla)

<A HREF="http://www.ucla.edu">UCLA</A>


In [90]:
vars(ucla)

{'onClick': None,
 'onMouseOut': None,
 'onMouseOver': None,
 'target': None,
 'text': 'UCLA',
 'url': 'http://www.ucla.edu'}

In [91]:
from IPython.display import HTML
HTML(str(ucla))

## 2nd version  makes it drastically easier for clients to create Href instances by using default arguments
If a function defines default arguments, then client code need not provide for such arguments.  If a function call does not provide a value for such arguments, then such arguments assume the default values.

In [92]:
def f(x, y=10): return x+y

In [93]:
f(10, 20)

30

In [94]:
f(30)

40

In [95]:
f(x=30)

40

In [96]:
f(y=40, x=50)

90

In [97]:
# f(y=20)

## 2nd version 

In [98]:
cat H_2.py

class Href:

    """The only difference between this and the previous version is that it
    uses default arguments.

    The definition of the initilizer remains hard to write and ugly,
    but it becomes so much easier to call, because client code can omit
    any arguments that it does not use.

    The structure of the Href objects created using both versions remain unchanged.

    Note that similar to Java, all Href instances have the same set
    and same number of attributes, even when some are not being used.

    Can this be avoided?  That is, is it possible for an instance to not have an
    attribute that it does not really need, such as target?

    """

    def __init__(self,
                 url,
                 text,
                 target=None,
                 onClick=None,
                 onMouseOver=None,
                 onMouseOut=None):

        self.url = url
        self.text = text
        self.target = target
        self.onClick = onClick 
        self.onM

In [99]:
run H_2.py

<A HREF="http://sandiego.edu">USD</A>
<A HREF="http://ucsd.edu" TARGET="_blank">UCSD</A>
{'text': 'USD', 'onClick': None, 'onMouseOver': None, 'target': None, 'onMouseOut': None, 'url': 'http://sandiego.edu'}
{'text': 'UCSD', 'onClick': None, 'onMouseOver': None, 'target': '_blank', 'onMouseOut': None, 'url': 'http://ucsd.edu'}


Notice how much relief the default arguments have brought the clients!  

Unfortunately, all Href instances still must possess all attributes that a Href can legally have, even though an instance uses only a handful.

In [100]:
print(len(vars(usd))); print(len(vars(ucsd)))

6
6


## Problems with the second version of Hrefgen

All Href instances have all the attributes that a Link should know about, but do not really use.  For example, `usd` has the attribute `target`, even though it does not use it.  `usd.target` holds a useless value `None`.  The big picture is that using such design, all HTMLElements collectively hold an enormous number of attributes that they do not use.  

### Question: Is it possible to make a HTMLElement have just those attributes it actually use?

See the following problem:

# Problem: 

**It is a fact that the great majority of USD students are US citizens**.  Write a `Student` class to (space-efficiently) model USD students so that `<student>.citizenship` evalutes to "US" (.e.g, `eric.citizenship` evaluates to `US`) for US citizens, but *mysteriously* such `Student`s do not need to have an attribute named `citizenship`.  

In [103]:
class Student:
    citizenship = 'US'
    def __init__(self, name, citizen='US'):
        self.name = name
        if citizen != 'US': # do not add citizenship attribute for US citizens
            self.citizenship = citizen

In [104]:
yusef = Student('Yuself', "Kuwait")        

In [105]:
vars(yusef)

{'citizenship': 'Kuwait', 'name': 'Yuself'}

In [108]:
eric = Student('Eric')

In [109]:
vars(eric)

{'name': 'Eric'}

In [134]:
eric.citizenship

'US'

In [110]:
pwd

'/Users/lubyliao/cs305'

In [137]:
cat H_3.py

"""In this third version, each instance has only attibutes that
it actually need.

An eye sore of this version is that the number of the if statements
grows linearly as the number of the arguments.  So, in the case of
the Table class, we might have 40+ if statements.

"""

class Href:

    target=None
    onClick=None
    onMouseOver=None
    onMouseOut=None

    def __init__(self,
                 url,
                 text,
                 target=None,
                 onClick=None,
                 onMouseOver=None,
                 onMouseOut=None):
        self.url = url
        self.text = text
        if target: self.target = target # Add target attribute only when it is used!
        if onClick: self.onClick = onClick # Add onClick attribute only when it is used!
        if onMouseOver:
            self.onMouseOver = onMouseOver
        if onMouseOut:
            self.onMouseOut = onMouseOut 

    def __str__(self):
        s = ['<A HREF="%s"' % self.url]
        if self.targe

In [147]:
%%writefile H3.py
"""In this third version, each instance has only attibutes that
it actually need.

An eye sore of this version is that the number of the if statements
grows linearly as the number of the arguments.  So, in the case of
the Table class, we might have 40+ if statements.

"""

class Href:

    target=None
    onClick=None
    onMouseOver=None
    onMouseOut=None

    def __init__(self,
                 url,
                 text,
                 target=None,
                 onClick=None,
                 onMouseOver=None,
                 onMouseOut=None):
        self.url = url
        self.text = text
        if target: self.target = target # Add target attribute only when it is used!
        if onClick: self.onClick = onClick # Add onClick attribute only when it is used!
        if onMouseOver:
            self.onMouseOver = onMouseOver
        if onMouseOut:
            self.onMouseOut = onMouseOut 

    def __str__(self):
        s = ['<A HREF="%s"' % self.url]
        if self.target: s.append(' TARGET="%s"' % self.target)
        if self.onClick: s.append(' onClick="%s"' % self.onClick)
        if self.onMouseOver: s.append(' onMouseOver="%s"' % self.onMouseOver)
        if self.onMouseOut: s.append(' onMouseOut="%s"' % self.onMouseOut)
        s.append('>%s</A>' % self.text)
        return ''.join(s)

ucsd = Href('UCSD', 'http://ucsd.edu', target='_blank')
harvard = Href('Harvard University', 'http://harvard.edu')
print(ucsd); print(harvard)
print(vars(ucsd)); print(vars(harvard)) 

Overwriting H3.py


In [148]:
run H3.py

<A HREF="UCSD" TARGET="_blank">http://ucsd.edu</A>
<A HREF="Harvard University">http://harvard.edu</A>
{'target': '_blank', 'text': 'http://ucsd.edu', 'url': 'UCSD'}
{'text': 'http://harvard.edu', 'url': 'Harvard University'}


## Problem with version 3 of Hrefgen

For each *optional* attribute, the body of `__init__` has a corresponding `if` statement to decide whether to add the attrbiute to `self`'s namespace.  For complicated HTMLElement, such long string of `if` would be unberable.  The solution is to force client code to provide **optional keyword arguments** as the following sinple example shows. 

In [117]:
def f(x, y, **kw):
    print(kw)

In [118]:
f(1,2)

{}


In [119]:
f(1,2,3)

TypeError: f() takes 2 positional arguments but 3 were given

In [122]:
f(y=3, x='abc', sex='M', age=3, hobby='Python')

{'hobby': 'Python', 'sex': 'M', 'age': 3}


In [123]:
cat H_4.py

"""This is a quantum leap from the previous version.  Note that the long
list of arguments in the definition of the initializer disappears, and
so does the equally long block of the if statements.

It is amazing that no matter how many attributes the class uses, the
size of its initializer remains a constant.
"""

class Href:

    target=None
    onClick=None
    onMouseOver=None
    onMouseOut=None

    def __init__(self,
                 url,
                 text,
                 **kw):

        self.url = url
        self.text = text

        for key in kw:
            setattr(self, key, kw[key])

    def __str__(self):
        s = ['<A HREF="%s"' % self.url]
        if self.target: s.append(' TARGET="%s"' % self.target)
        if self.onClick: s.append(' onClick="%s"' % self.onClick)
        if self.onMouseOver: s.append(' onMouseOver="%s"' % self.onMouseOver)
        if self.onMouseOut: s.append(' onMouseOut="%s"' % self.onMouseOut)
        s.append('>%s</A>' % self.text)
     

In [124]:
run H_4.py

<A HREF="UCSD" TARGET="_blank">http://ucsd.edu</A>
<A HREF="Harvard University">http://harvard.edu</A>
{'target': '_blank', 'text': 'http://ucsd.edu', 'url': 'UCSD'}
{'text': 'http://harvard.edu', 'url': 'Harvard University'}


In [131]:
ucla = Href(url='http://ucla.edu', text='UCLA', target='_blank', hobby='beer',
           likes='football')

In [132]:
print(ucla)

<A HREF="http://ucla.edu" TARGET="_blank">UCLA</A>


In [133]:
vars(ucla)

{'hobby': 'beer',
 'likes': 'football',
 'target': '_blank',
 'text': 'UCLA',
 'url': 'http://ucla.edu'}

## Above example shows that version 4 is not valid

because it allows clients to introduce meaningless attributes, such as `hobby` for a `Href`.  This problem can be fixed by a one-liner.

In [1]:
pwd

'/Users/lubyliao'

In [2]:
cd cs305

/Users/lubyliao/cs305


In [6]:
cat H_5.py

"""The previous version 4 is erroneous because it allows clients to
provide meaningless attributes such as sex, age, etc.  

This version checks to make sure that meaningless attributes, if
provided, causes exception and exit the initializer.

The idea is that at the point when we are to add the client-provided
optional attributes beyond the required url and text, the namespaces
of the instance self and the class are already populated with all
meaningful names.  Therefore, the initilizer simply checks to see
whether the provided attribute (through the keyword argument) is in
the self's or class's namespace with getattr(self, key).

"""

class Href:

    target=None
    onClick=None
    onMouseOver=None
    onMouseOut=None

    def __init__(self,
                 url,
                 text,
                 **kw):

        self.url = url
        self.text = text

        for key in kw:
            try:
                getattr(self, key)
                setattr(self, key, kw[key])
      

## To emphasie: the key point of Version 5 is this
When we are about to use client-provided keyword arguments to create instance variables for the instance, the class and the instance are already filled with all possible meaningful names.

## Next, think beyond Hrefgen and notice the code duplication

1. HTMLgen library will contain not just Href class, but also Table class, Form class, Input class, Select class, and many many more (see HTMLgen.py)
1. Each such class' `__init__` contains the same piece of code

        for key in kw:
            try:
                getattr(self, key)
                setattr(self, key, kw[key])
            except:
                raise KeyError("You provide an argument %s that makes no sense" % key)
                
1. Furthermore, this piece of code is completely generic and devoid of context, so

1. We should avoid duplicate this code across all HTMLElement classes, so
1. We should factor this code into a superclass HTMLElement, as in the next version.


In [7]:
cat H_6.py

"""
Factor generic code from subclasses such as Href, Table, etc into a superclass
to achieves code reuse and avoid code duplication.
"""

class HTMLElement:
    def __init__(self, **kw):
        for attr in kw:
            try:
                getattr(self, attr)
                setattr(self, attr, kw[attr])
            except:
                raise KeyError("Unsupported argument %s!" % attr)

class Href(HTMLElement):

    target = None
    onClick = None
    onMouseOver = None
    onMouseOut = None

    def __init__(self,
                 url='',
                 text='',
                 **kw):
        
        self.url = url
        self.text = text
        HTMLElement.__init__(self, **kw)

    def __str__(self):
        s = ['<A HREF="%s"' % self.url]
        if self.target: s.append(' TARGET="%s"' % self.target)
        if self.onClick: s.append(' onClick="%s"' % self.onClick)
        if self.onMouseOver: s.append(' onMouseOver="%s"' % self.onMouseOver)
        if self.onMouseOut

In [8]:
run H_6.py

<A HREF="http://www.sandiego.edu" TARGET="_blank">USD</A>
<TABLE border=2 cellpadding=4 cellspacing=1 width="100%">
<TR Align=center> <TH ColSpan=1>A</TH><TH ColSpan=1>B</TH><TH ColSpan=1>C</TH></TR>
<TR><TD Align=left >1</TD>
<TD Align=left >2</TD>
<TD Align=left >3</TD>
</TR>
<TR><TD Align=left >4</TD>
<TD Align=left >5</TD>
<TD Align=left >6</TD>
</TR>
<TR><TD Align=left >7</TD>
<TD Align=left >8</TD>
<TD Align=left >9</TD>
</TR>
</TABLE><P>



## Next: Modeling a polynomial.  Please read `polynomial.py` on the github 

## An important aside on the use of /tmp on holycow

Note that when I experimented with `form_gen.py`, I put it in `holycow:/tmp`.  One benefit is that the system will delete it automatically, so I don't have to worry about leaving such garbage behind.  An added benefit is that it is publicly readable so

> **before the system delete it, anyone can read it**
  
If I had put it under my home directory, then the file is not accessible by anyone else.